# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686570


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:29,  1.05s/it]

Rendering models:  10%|█         | 3/30 [00:09<01:17,  2.87s/it]

Rendering models:  13%|█▎        | 4/30 [00:10<01:02,  2.41s/it]

Rendering models:  17%|█▋        | 5/30 [00:11<00:48,  1.96s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:28<00:02,  1.09it/s]

Rendering models:  93%|█████████▎| 28/30 [00:29<00:01,  1.19it/s]

Rendering models:  97%|█████████▋| 29/30 [00:30<00:00,  1.17it/s]

Rendering models: 100%|██████████| 30/30 [00:30<00:00,  1.24it/s]

bimlesh                               0.000874
smithkurtis                           0.000972
RockyGeo                              0.000837
not-logged-in-bc5fdb9ea8d611e60b8b    0.000617
not-logged-in-485863b819b2c7db6f96    0.000646
amber_9                               0.011898
CThomas                               0.002121
ElisabethB                            0.000668
Camelron                              0.000709
theetick                              0.003149
not-logged-in-879a4157a85361e9a325    0.001017
sn346808                              0.021779
Arbitrationer                         0.001376
LeeJoseph                             0.005544
not-logged-in-518ca9e22e894704402a    0.001940
not-logged-in-ffc5c906399855f5ef0f    0.000851
KrisKeogh                             0.001022
skotecha                              0.000644
connorstone628                        0.000612
not-logged-in-e23c11d8538d9848a6a5    0.006686
not-logged-in-14d09f70263c1fe15cc3    0.001225
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())